# Algo de novo em Python 3

Veja estes exemplos em Python 2:

```python
>>> L = [0, 1, 2]
>>> zip(L, 'ABC')
[(0, 'A'), (1, 'B'), (2, 'C')]
>>> map(lambda x: x*10, L)
[0, 10, 20]
>>> filter(None, L)
[1, 2]
```

Agora os mesmos comandos em Python 3:

In [1]:
L = [0, 1, 2]
zip(L, 'ABC')

In [2]:
map(lambda x: x*10, L)

In [3]:
filter(None, L)

É como diz o Hélio Meira Lins: quase nada em Python 3 devolve uma lista.

Para construir listas, use o construtor `list`!

In [4]:
list(zip(L, 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [5]:
list(map(lambda x: x*10, L))

[0, 10, 20]

In [6]:
list(filter(None, L))

[1, 2]

Porém esses últimos exemplos eu faria assim:

In [7]:
[x*10 for x in L]

[0, 10, 20]

In [8]:
[x for x in L if x]

[1, 2]

E a versão *lazy* (preguiçosa), eu faria assim:

In [9]:
(x*10 for x in L)

<generator object <genexpr> at 0x105360af0>

A novidade é uso o ainda mais extensivo de geradores em Python 3.

# Como funciona um objeto gerador

Objetos geradores são iteradores: oferecem um método `__next__` que, quando acionado pela função embutida `next`, produzem o próximo item de uma série. A série pode ser limitada ou ilimitada.

In [10]:
z = zip(L, 'ABC')
z

In [11]:
next(z)

(0, 'A')

In [12]:
next(z)

(1, 'B')

In [13]:
next(z)

(2, 'C')

## Quando esgotado, um gerador levanta `StopIteration`

In [14]:
# next(z)

Não há como reiniciar um gerador. Se for preciso percorrer duas vezes, é preciso jogá-lo fora e começar outro:

In [15]:
z = zip(L, 'ABC')
next(z)

(0, 'A')

# Trem iterável

In [19]:
class Trem(object):

    def __init__(self, vagoes):
        self.vagoes = vagoes

    def __iter__(self):
        return IteradorTrem(self.vagoes)

class IteradorTrem(object):

    def __init__(self, vagoes):
        self.atual = 0
        self.ultimo_vagao = vagoes - 1

    def __next__(self):
        if self.atual <= self.ultimo_vagao:
            self.atual += 1
            return 'vagão #%s' % (self.atual)
        else:
            raise StopIteration()

In [24]:
t = Trem(3)
for vagão in t:
    print(vagão)

vagão #1
vagão #2
vagão #3


In [25]:
it = iter(t)
it

In [26]:
next(it)

'vagão #1'

In [27]:
next(it)

'vagão #2'

In [28]:
next(it)

'vagão #3'

In [29]:
next(it)

StopIteration: 

In [30]:
class Trem(object):

    def __init__(self, vagoes):
        self.vagoes = vagoes

    def __iter__(self):
        for i in range(self.vagoes):
            yield 'vagão #%s' % (i+1)

In [32]:
t = Trem(3)
it = iter(t)
it

<generator object __iter__ at 0x10539f4c0>

In [33]:
next(it), next(it), next(it)

('vagão #1', 'vagão #2', 'vagão #3')

In [34]:
next(it)

StopIteration: 

# Gerador de Fibonacci

In [89]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [90]:
fib = fibonacci()
for i in range(10):
    print(next(fib))

0
1
1
2
3
5
8
13
21
34


In [93]:
def fibonacci(n):
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b

In [94]:
list(fibonacci(10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [97]:
list(fibonacci(100))[-3:]

[83621143489848422977, 135301852344706746049, 218922995834555169026]

# Gerador de Progressão Aritmética

In [52]:
def progressão_aritmética(incremento, *, início=0, término=None):
    infinita = término is None
    índice = 0
    resultado = início + incremento * índice
    while infinita or resultado < término:
        yield resultado
        índice += 1
        resultado = início + incremento * índice

In [53]:
pa = progressão_aritmética(.1)
next(pa), next(pa), next(pa), next(pa), next(pa)

(0.0, 0.1, 0.2, 0.30000000000000004, 0.4)

In [54]:
from decimal import Decimal
pa = progressão_aritmética(Decimal('.1'))
next(pa), next(pa), next(pa), next(pa), next(pa)

(Decimal('0.0'),
 Decimal('0.1'),
 Decimal('0.2'),
 Decimal('0.3'),
 Decimal('0.4'))

In [55]:
from fractions import Fraction
pa = progressão_aritmética(Fraction(1, 10))
next(pa), next(pa), next(pa), next(pa), next(pa)

(Fraction(0, 1),
 Fraction(1, 10),
 Fraction(1, 5),
 Fraction(3, 10),
 Fraction(2, 5))

In [57]:
pa = progressão_aritmética(.5, início=1, término=3)
list(pa)

[1.0, 1.5, 2.0, 2.5]

In [58]:
pa = progressão_aritmética(1/3, término=1)
list(pa)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [59]:
pa = progressão_aritmética(2, término=10)
list(pa)

[0, 2, 4, 6, 8]

## Gerador de PA simplificado com `itertools`

In [62]:
import itertools


def progressão_aritmética(incremento, *, início=0, término=None):
    primeiro = type(início + incremento)(início)
    gen_pa = itertools.count(primeiro, incremento)
    if término is not None:
        gen_pa = itertools.takewhile(lambda n: n < término, gen_pa)
    return gen_pa

In [63]:
pa = progressão_aritmética(.1)
next(pa), next(pa), next(pa), next(pa), next(pa)

(0.0, 0.1, 0.2, 0.30000000000000004, 0.4)

In [64]:
pa = progressão_aritmética(Decimal('.1'))
next(pa), next(pa), next(pa), next(pa), next(pa)

(Decimal('0'), Decimal('0.1'), Decimal('0.2'), Decimal('0.3'), Decimal('0.4'))

In [65]:
pa = progressão_aritmética(2, término=10)
list(pa)

[0, 2, 4, 6, 8]

# Iterador por palavras

In [73]:
import re
import reprlib

RE_WORD = re.compile('\w+')


class Frase:

    def __init__(self, texto):
        self.texto = texto  # <1>

    def __repr__(self):
        return 'Frase(%s)' % reprlib.repr(self.texto)

    def __iter__(self):
        for match in RE_WORD.finditer(self.texto):  # <2>
            yield match.group()  # <3>

In [77]:
quadrilha = '''João amava Teresa que amava Raimundo
que amava Maria que amava Joaquim que amava Lili
que não amava ninguém.'''  # Carlos Drummond de Andrade

In [78]:
versos = Frase(quadrilha)
versos

Frase('João amava T...mava ninguém.')

In [79]:
for palavra in versos:
    print(palavra)

João
amava
Teresa
que
amava
Raimundo
que
amava
Maria
que
amava
Joaquim
que
amava
Lili
que
não
amava
ninguém
